In [1]:
##Imports
from Resnext50 import Resnext50
import torch
import BigEarthNetConfig
from BigEarthNetDataset import BigEarthNetDataset
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, IterableDataset
import numpy as np
import warnings
from tqdm import tqdm


In [2]:

# Initialize the model

device = torch.device("cpu")

model = Resnext50(len(BigEarthNetConfig.BigEarthNetlabels))
# Switch model to the training mode
model.train()
model.load_state_dict(torch.load("checkpoint.pt"))
model.eval()

Resnext50(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): 

In [3]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
testDataSet = BigEarthNetDataset(
    data_path='./data/test', label_filename="labels.csv", transform=test_transform)

testDataLoader = DataLoader(testDataSet, batch_size=32, shuffle=True, num_workers=4)

In [5]:
from utils import calculate_metrics


warnings.filterwarnings('ignore')

epoch = 0
iteration = 0
test_freq = 7
save_freq = 1

resultF1 = 0

model.eval()
with torch.no_grad():
    model_result = []
    targets = []
    for index,imgsAndBatchTargets in enumerate(tqdm(testDataLoader)):
        imgs, batch_targets = imgsAndBatchTargets
        imgs = imgs.to(device)
        model_batch_result = model(imgs)
        model_result.extend(model_batch_result.cpu().numpy())
        targets.extend(batch_targets.cpu().numpy())

result = calculate_metrics(
    np.array(model_result), np.array(targets))
resultF1 = result.get("macro/f1")
print("epoch:{:2d} iter:{:3d} test: "
                  "micro f1: {:.3f} "
                  "macro f1: {:.3f} "
                  "samples f1: {:.3f}".format(epoch, iteration,
                                              result['micro/f1'],
                                              result['macro/f1'],
                                              result['samples/f1']))

100%|██████████| 16/16 [00:26<00:00,  1.64s/it]

epoch: 0 iter:  0 test: micro f1: 0.611 macro f1: 0.099 samples f1: 0.561
